In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Tham khảo:
https://thorpham.github.io/blog/2018/04/06/Nh%E1%BA%ADn-d%E1%BA%A1ng-ch%E1%BB%AF-s%E1%BB%91-vi%E1%BA%BFt-tay-v%E1%BB%9Bi-sklearn-v%C3%A0-opencv/


In [0]:
import time
import imutils
import dlib
import cv2
import pickle
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *
from keras import applications
import tensorflow as tf
from keras import backend as K
import h5py


In [0]:
def convnet_model_():
    vgg_model = applications.VGG16(weights=None, include_top=False, input_shape=(221, 221, 3))
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Lambda(lambda x_: K.l2_normalize(x,axis=1))(x)
#     x = Lambda(K.l2_normalize)(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def deep_rank_model():
    convnet_model = convnet_model_()

    first_input = Input(shape=(221, 221, 3))
    first_conv = Conv2D(96, kernel_size=(8,8), strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(first_conv)
    first_max = Flatten()(first_max)
#     first_max = Lambda(K.l2_normalize)(first_max)
    first_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(first_max)

    second_input = Input(shape=(221, 221, 3))
    second_conv = Conv2D(96, kernel_size=(8,8), strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7), strides=(4,4), padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(second_max)
#     second_max = Lambda(K.l2_normalize)(second_max)
                       
    merge_one = concatenate([first_max, second_max])
    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    emb = Dense(128)(emb)
    l2_norm_final = Lambda(lambda x: K.l2_normalize(x, axis=1))(emb)
#     l2_norm_final = Lambda(K.l2_normalize)(emb)
                        
    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model


In [21]:
#Load model
deep_rank_model = deep_rank_model()
deep_rank_model.summary()
deep_rank_model.load_weights('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/triplet_weight1.hdf5')

#Load all vector embedding LFW of my model
with open('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/embs128.pkl', 'rb') as f:
    embs128 = pickle.load(f)
with open('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/LFW_128_HA_labels.tsv', 'r') as f:
    names = f.readlines()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 221, 221, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 221, 221, 64) 1792        input_16[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 221, 221, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 110, 110, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

ResourceExhaustedError: ignored

In [0]:


#Read image
image = cv2.imread('/home/pham.hoang.anh/Desktop/1.jpg')
image = imutils.resize(image, width=800)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

hog_face_detector = dlib.get_frontal_face_detector()
# cnn_face_detector = dlib.cnn_face_detection_model_v1('/Users/phamhoanganh/Desktop/mmod_human_face_detector.dat')

# start = time.time()
faces_hog = hog_face_detector(image, 1)
# end = time.time()
# print("Hog + SVM Execution time: " + str(end-start))